# Specifying a Data Type

In [1]:
import d2ltvm
import sys
print(sys.path)
import tvm
from tvm import te
import numpy as np

n = 100
def tvm_vector_add(dtype):
    A = te.placeholder((n,), name='A', dtype=dtype)
    B = te.placeholder((n,), name='B', dtype=dtype)
    C = te.compute(A.shape, lambda i: A[i] + B[i])
    print('expression dtype:',A.dtype, B.dtype, C.dtype)
    s = te.create_schedule(C.op)
    return tvm.build(s, [A, B, C])

mod = tvm_vector_add('int32')

successd...

['/staff/qiaoliang/ACSA科研项目/tvm_learning/ch2_expressions', '/staff/qiaoliang/ACSA科研项目/TVM/tvm/python', '/staff/qiaoliang/ACSA科研项目/tvm_learning/ch2_expressions', '/staff/qiaoliang/ACSA科研项目/tvm_learning', '/staff/qiaoliang/anaconda3/envs/tvm/lib/python38.zip', '/staff/qiaoliang/anaconda3/envs/tvm/lib/python3.8', '/staff/qiaoliang/anaconda3/envs/tvm/lib/python3.8/lib-dynload', '', '/staff/qiaoliang/anaconda3/envs/tvm/lib/python3.8/site-packages']
expression dtype: int32 int32 int32


In [2]:
def test_mod(mod, dtype):
    a, b, c = d2ltvm.get_abc(n, lambda x: tvm.nd.array(x.astype(dtype)))
    print("tensor dtype:", a.dtype, b.dtype, c.dtype)
    mod(a, b, c)
    np.testing.assert_equal(c.asnumpy(), a.asnumpy() + b.asnumpy())

test_mod(mod, 'int32')

tensor dtype: int32 int32 int32


In [3]:
for i in ('float16', 'float64', 'int8', 'int16', 'int64'):
    mod = tvm_vector_add(i)
    test_mod(mod, i)

expression dtype: float16 float16 float16
tensor dtype: float16 float16 float16
expression dtype: float64 float64 float64
tensor dtype: float64 float64 float64
expression dtype: int8 int8 int8
tensor dtype: int8 int8 int8
expression dtype: int16 int16 int16
tensor dtype: int16 int16 int16
expression dtype: int64 int64 int64
tensor dtype: int64 int64 int64


# Converting Elements Data Types

In [4]:
def tvm_vector_add_2(shape, dtype):
    '''te.placeholder ---> astype(); tvm.nd.array --×-- astype()
    '''
    A = te.placeholder((n,), name='a')
    B = te.placeholder((n,), name='b')
    C = te.compute(A.shape, lambda i: A[i].astype(dtype) + B[i].astype(dtype))
    print('expression type:', A.dtype, B.dtype, C.dtype)
    s = te.create_schedule(C.op)
    return tvm.build(s, [A, B, C])

In [5]:
def test_mod_2(mod, dtype):
    a, b, c = d2ltvm.get_abc(n)
    a_tvm, b_tvm = tvm.nd.array(a), tvm.nd.array(b)
    c_tvm = tvm.nd.array(c.astype(dtype))
    print('tensor dtype:', a_tvm.dtype, b_tvm.dtype, c_tvm.dtype)
    mod(a_tvm, b_tvm, c_tvm)
    np.testing.assert_equal(c_tvm.asnumpy(), a.astype(dtype) + b.astype(dtype))

for i in ('float16', 'float64', 'int8', 'int16', 'int64'):
    mod = tvm_vector_add_2(n, i)
    test_mod_2(mod, i)

expression type: float32 float32 float16
tensor dtype: float32 float32 float16
expression type: float32 float32 float64
tensor dtype: float32 float32 float64
expression type: float32 float32 int8
tensor dtype: float32 float32 int8
expression type: float32 float32 int16
tensor dtype: float32 float32 int16
expression type: float32 float32 int64
tensor dtype: float32 float32 int64


# Summary
1.We can specify the data type by dtype when creating TVM placeholders.(define placeholder)

2.The data type of a tensor element can be cast by astype in TVM compute.(define compute op)